In [ ]:
# Libraries
import pandas as pd
import numpy as np
from functools import reduce
from tqdm import tqdm





In [3]:
# Load training data
cos_train=pd.read_pickle('../Features/cos_sim_features_train.pickle')
lsm_train=pd.read_pickle('../Features/LSM_features_train.pickle')
phq9_train=pd.read_pickle('../Features/PHQ9_features_train.pickle')
re_train=pd.read_pickle('../Features/RE_features_train.pickle')
re_train=re_train.dropna(subset=['re']).copy()

# Load testing data
cos_test=pd.read_pickle('../Features/cos_sim_features_test.pickle')
lsm_test=pd.read_pickle('../Features/LSM_features_test.pickle')
phq9_test=pd.read_pickle('../Features/PHQ9_features_test.pickle')
re_test=pd.read_pickle('../Features/RE_features_test.pickle')
re_test=re_test.dropna(subset=['re']).copy()

print(len(phq9_train))
print(len(re_train))
print(len(lsm_train))
print(len(cos_train))

133430
114280
104057
104231


In [ ]:
# Checking the data
print('-'*25+'Colums in Training data'+'-'*25)
print(cos_train.columns)
print(lsm_train.columns)
print(phq9_train.columns)
print(re_train.columns)
print('-'*25+'Colums in Testing data'+'-'*25)
print(cos_test.columns)
print(lsm_test.columns)
print(phq9_test.columns)
print(re_test.columns)
datasets = {
    "cos_train": cos_train,
    "lsm_train": lsm_train,
    "phq9_train": phq9_train,
    "re_train": re_train
}

print("\n" + "="*60)
print("  Depression label distribution in TRAIN datasets")
print("="*60)

for name, df in datasets.items():
    print(f"\n{name}")
    print(df['Depression'].value_counts(dropna=False))  


print("\n" + "="*60)
print(" Check duplicated rows in TRAIN datasets")
print("="*60)

for name, df in datasets.items():
    dup_df = df[df.duplicated()]  
    print(f"\n {name}: duplicated rows = {dup_df.shape[0]}")
    
for name, df in datasets.items():


    df = df.copy()
    df['key'] = df.apply(
        lambda r: r['submission_id'] if r.get('type')=='submission' else r.get('comment_id'),
        axis=1
    )

  
    dup_keys = df[df['key'].duplicated(keep=False)].sort_values('key')

    print(f"\n {name}")
    print(f"dup keysubmission_id/comment_id: {dup_keys['key'].nunique()}")
    print(f": {dup_keys.shape[0]}")


    if not dup_keys.empty:
        print(" Sample duplicated keys:")
        print(dup_keys[['key','type','user_id','created_utc']].head(10))


-------------------------Colums in Training data-------------------------
Index(['user_id', 'target', 'type', 'title', 'body', 'created_utc',
       'submission_id', 'parent_id', 'comment_id', 'Depression', 'parent',
       'sim', 'max_sim_so_far', 'min_sim_so_far', 'max_gap_sim_so_far',
       'delta_sim', 'rolling_std_sim'],
      dtype='object')
Index(['user_id', 'body', 'type', 'Depression', 'created_utc', 'comment_id',
       'submission_id', 'parent_body', 'LSM_mean', 'social_mean',
       'positive_mean', 'negative_mean', 'first_person_singular',
       'second_person', 'third_person_singular', 'third_person_plural',
       'cognitive_process', 'perceptual_process', 'LSM_mean_max_so_far',
       'LSM_mean_min_so_far', 'LSM_mean_max_gap_so_far', 'LSM_mean_delta',
       'LSM_mean_rolling_std', 'social_mean_max_so_far',
       'social_mean_min_so_far', 'social_mean_max_gap_so_far',
       'social_mean_delta', 'social_mean_rolling_std',
       'positive_mean_max_so_far', 'positive_

In [ ]:
import pandas as pd
from tqdm import tqdm

# set phq9 table as base
train = phq9_train.copy()
test  = phq9_test.copy()

# Adding other source 
sources_train = {"cos":cos_train, "lsm":lsm_train, "re":re_train}
sources_test  = {"cos":cos_test,  "lsm":lsm_test,  "re":re_test}

# extreact feature cols
def get_feature_cols(df):
    return [c for c in df.columns if c not in 
            ['user_id','target','type','title','body','created_utc',
             'submission_id','parent_id','comment_id','Depression','parent']]

# all features for cos, lsm and re
cos_cols = get_feature_cols(cos_train)
lsm_cols = get_feature_cols(lsm_train)
re_cols  = get_feature_cols(re_train)

ALL_FEATURE_COLS = cos_cols + lsm_cols + re_cols

#  add all features in phq9 and defult =-1
for col in ALL_FEATURE_COLS:
    train[col] = -1
    test[col]  = -1

# fill added feature cols by key
def fill_features(phq_df, sources):
    for i in tqdm(range(len(phq_df))):
        row = phq_df.iloc[i]

        # submission -> using submission_id as key
        if row["type"]=="submission":
            sid = row["submission_id"]
            for name, df in sources.items():
                match = df[(df["type"]=="submission") & (df["submission_id"]==sid)]
                if len(match)>0:
                    phq_df.iloc[i, phq_df.columns.isin(get_feature_cols(df))] = match[get_feature_cols(df)].values[0]

        # comment → using comment_id as key
        elif row["type"]=="comment":
            cid = row["comment_id"]
            for name, df in sources.items():
                match = df[(df["type"]=="comment") & (df["comment_id"]==cid)]
                if len(match)>0:
                    phq_df.iloc[i, phq_df.columns.isin(get_feature_cols(df))] = match[get_feature_cols(df)].values[0]
    return phq_df


print("==== Filling TRAIN features ====")
train = fill_features(train, sources_train)

print("==== Filling TEST features ====")
test  = fill_features(test, sources_test)


train.to_pickle("/u50/zhanh279/4Z03/jupyter/Features/final_training_dataset.pickle")
test.to_pickle("/u50/zhanh279/4Z03/jupyter/Features/final_testing_dataset.pickle")


==== Filling TRAIN features ====


  0%|          | 0/133430 [00:00<?, ?it/s]/tmp/ipykernel_983382/3984813263.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0003346651404475172' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  phq_df.iloc[i, phq_df.columns.isin(get_feature_cols(df))] = match[get_feature_cols(df)].values[0]
/tmp/ipykernel_983382/3984813263.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.147467931111654' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  phq_df.iloc[i, phq_df.columns.isin(get_feature_cols(df))] = match[get_feature_cols(df)].values[0]
/tmp/ipykernel_983382/3984813263.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.147133265971206' has dtype incomp

==== Filling TEST features ====


  0%|          | 0/45762 [00:00<?, ?it/s]/tmp/ipykernel_983382/3984813263.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.753734827041626' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  phq_df.iloc[i, phq_df.columns.isin(get_feature_cols(df))] = match[get_feature_cols(df)].values[0]
/tmp/ipykernel_983382/3984813263.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.753734827041626' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  phq_df.iloc[i, phq_df.columns.isin(get_feature_cols(df))] = match[get_feature_cols(df)].values[0]
/tmp/ipykernel_983382/3984813263.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.753734827041626' has dtype incompatibl

🚀 完成 — 行级逐特征覆盖版合表已生成。
